<a href="https://colab.research.google.com/github/Ahmed-Elbagoury/my_transformers/blob/main/my_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# PyTorch
import torch
import math
import torch.nn.functional as F
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl

In [68]:
# TODO: Your implementation goes in here
def scaled_dot_product(q, k, v, mask=None):
  attn_logits = q @ k.transpose(-2, -1) / math.sqrt(q.shape[-1])
  if mask is not None:
    attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
  att = F.softmax(attn_logits, dim=-1)
  return att @ v, att

In [69]:
# TODO: do not change this.
# It's copied from https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.ipynb
def ref_scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [70]:
# Test cases are from https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.ipynb
seq_len, d_k = 3, 2
pl.seed_everything(42)
q = torch.randn(seq_len, d_k)
k = torch.randn(seq_len, d_k)
v = torch.randn(seq_len, d_k)

ref_values, ref_attention = ref_scaled_dot_product(q, k, v)
values, attention = scaled_dot_product(q, k, v)

assert torch.allclose(ref_values, values)
assert torch.allclose(ref_attention, attention)

INFO:lightning_fabric.utilities.seed:Seed set to 42
